# LangChain Fundamentals
***
## Table of Contents
***

## 1. Introduction

## 2. Environmental Variables
Firstly, environmental variables need to be configured. These variables, especially API keys should never be hardcoded or made visible to others. The [python-dotenv](https://pypi.org/project/python-dotenv/) libray makes it straightforward to securely access variables set in a `.env` file.

In [16]:
import os
from getpass import getpass

try:
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    raise ImportError("Error: 'python-dotenv' not installed")

`os.environ` is a dictionary-like object representing the environment variables of the current process. It allows users to assign new values using the syntax: `os.environ['some_key'] = 'some_value'`

In [17]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API key: "
)


## 3. Using Language Model
### Loading Model
There are multiple approaches to loading language models:
1. Use `init_chat_model` from `langchain.chat_models`, with specified model name and provider. 
2. Specify the model's integration package first, then call the appropriate method to initialise the model.

In [18]:
from langchain.chat_models import init_chat_model
from langchain_openai import ChatOpenAI


model_name = "gpt-4o-mini"

# 1. Simpler call
# model = init_chat_model(model=model_name, model_provider="openai", temperature=0.8)

# 2. Directly use of the integration package
model = ChatOpenAI(model=model_name, temperature=0.8)

### Interacting with Language Model
For a simple call, we can pass `messages` to the `.invoke` method. The list of message objects (`messages`) can be categorised into three parts:
- SystemMessage: Text that guides or determines AI's behaviour or actions.
- HumanMessage: Input given by a user.
- AIMessage: Output generated by the model.

In [19]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Translate the following message from English into French"),
    HumanMessage(content="Today is a beautiful day"),
]

model.invoke(input=messages)

AIMessage(content="Aujourd'hui est une belle journée.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 24, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-C6wvckRkUZyPJFsdpWO59d37vBXVT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ac8ea802-78be-41c5-8a46-00a85cacb89c-0', usage_metadata={'input_tokens': 24, 'output_tokens': 7, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Prompt Template
A prompt template provides a structured way of creating inputs for language models where parts of the prompt can be dynamically changed based on context or user input.

Prompts in LangChain can be split into three components:
- System Prompt: Gives instructions or a personality to the LLM model. This prompt determines the behaviour or characteristics of the model.
- User Prompt: Input given by a user.
- AI Prompt: Output generated by the model.

In [20]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_prompt = SystemMessagePromptTemplate.from_template(
    template="You are an AI translater. Translate text from English to {language}",
    input_variables=["language"],
)

user_prompt = HumanMessagePromptTemplate.from_template(
    template="""Your task is to translate a text. The text to be translated is:
    ---
    {text}
    ---
    Output only the translated text, no other explanation or text should be provided
    """,
    input_variables=["text"],  # Define a variable
)

text_in_english = """
A croissant is a French Viennoiserie in a crescent shape made from a laminated yeast dough that sits between a bread and a puff pastry.
"""

target_language = "French"

Let's display the formatted user prompt after inserting a value into the `text` parameter:

In [21]:
print(user_prompt.format(text=text_in_english))

content='Your task is to translate a text. The text to be translated is:\n    ---\n    \nA croissant is a French Viennoiserie in a crescent shape made from a laminated yeast dough that sits between a bread and a puff pastry.\n\n    ---\n    Output only the translated text, no other explanation or text should be provided\n    ' additional_kwargs={} response_metadata={}


After defining system and user prompts, we can merge them into a full chat prompt using `ChatPromptTemplate`:

In [22]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

ChatPromptTemplate adds a prefix indicating a role of each message (e.g., System:, Human: or AI:).

In [23]:
print(prompt_template.format(text=text_in_english, language=target_language))

System: You are an AI translater. Translate text from English to French
Human: Your task is to translate a text. The text to be translated is:
    ---
    
A croissant is a French Viennoiserie in a crescent shape made from a laminated yeast dough that sits between a bread and a puff pastry.

    ---
    Output only the translated text, no other explanation or text should be provided
    


Using **L**ang**C**hain **E**xpression **L**anguage (LCEL), we can construct a chain that links the user input, prompt templates, the model and the output in a sequence:

`input | prompt template | model | output`

This pipeline enables smooth data flow, where the user input is formatted by the prompt template, passed to the model for inference, and the model's response is captured as the output.

Note that lambda expressions are required to access prompt template variables. These variables are stored within input dictionaries or complex objects, thus the lambdas explicity extract or map the necessary fields from these data structures to ensure the correct values are passed to each stage in the chain.

In [24]:
chain = (
    {"text": lambda x: x["text"], "language": lambda x: x["language"]}
    | prompt_template
    | model
    | {"translated_text": lambda x: x.content}
)

chain.invoke({"text": text_in_english, "language": target_language})

{'translated_text': "Un croissant est une viennoiserie française en forme de croissant, réalisée à partir d'une pâte à base de levure laminée qui se situe entre une pâte à pain et une pâte feuilletée."}